In [1]:
import pandas as pd

from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

import ast
from tqdm import tqdm
tqdm.pandas()

2022-11-24 07:18:01.861997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 07:18:03.012608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 07:18:03.012644: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-24 07:18:03.115273: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-24 07:18:06.153366: W tensorflow/stream_executor/pla

In [2]:
df = pd.read_csv("./../data/processed/data_10.csv")

df['Text'] = df['Text'].progress_apply(lambda x : ast.literal_eval(x))
df['Tags'] = df['Tags'].progress_apply(lambda x : ast.literal_eval(x))

df['Text'] = df['Text'].progress_apply(lambda x : ' '.join(map(str, x)))
df['Tags'] = df['Tags'].progress_apply(lambda x : ' '.join(map(str, x)))

100%|████████████████████████████████| 44296/44296 [00:00<00:00, 1191601.24it/s]


In [67]:
df.shape

(44296, 2)

In [4]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(100, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
#     model.add(Dense(100, input_dim=100, kernel_initializer='he_uniform', activation='relu', ))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [6]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true', min_df=1)
vectorizer.fit(df['Tags']) #tags

y = vectorizer.transform(df['Tags']).toarray()

tags_dict = vectorizer.vocabulary_


tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['Text']).toarray()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
n_inputs, n_outputs = X.shape[1], y.shape[1]

model = get_model(n_inputs, n_outputs)
# fit model
model.fit(X_train, y_train, verbose=0, epochs=100)
# make a prediction on the test set
yhat = model.predict(X_test)
yhat = yhat.round()

2022-11-24 07:19:14.409967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 07:19:14.485488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-24 07:19:14.485582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-24 07:19:14.485652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-24 07:19:14.485717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

 20/277 [=>............................] - ETA: 1s

2022-11-24 07:56:12.998715: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2370865120 exceeds 10% of free system memory.


277/277 [==============================] - 2s 6ms/step


In [8]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.89      0.79      0.84      1102
           1       0.67      0.41      0.51       592
           2       0.54      0.60      0.57      1501
           3       0.68      0.60      0.64       644
           4       0.88      0.79      0.84       640
           5       0.72      0.66      0.69      1319
           6       0.58      0.44      0.50      1209
           7       0.77      0.61      0.68       995
           8       0.79      0.69      0.74      1314
           9       0.86      0.78      0.82       636

   micro avg       0.72      0.64      0.67      9952
   macro avg       0.74      0.64      0.68      9952
weighted avg       0.72      0.64      0.67      9952
 samples avg       0.71      0.67      0.68      9952



/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.89      0.83      0.86      1102
           1       0.72      0.47      0.57       592
           2       0.62      0.65      0.63      1501
           3       0.69      0.62      0.65       644
           4       0.86      0.81      0.83       640
           5       0.75      0.71      0.73      1319
           6       0.71      0.50      0.58      1209
           7       0.81      0.68      0.74       995
           8       0.83      0.74      0.79      1314
           9       0.90      0.81      0.85       636

   micro avg       0.77      0.68      0.72      9952
   macro avg       0.78      0.68      0.72      9952
weighted avg       0.77      0.68      0.72      9952
 samples avg       0.77      0.71      0.73      9952



/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
# 1 layer, 100 units

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86      1102
           1       0.71      0.45      0.55       592
           2       0.59      0.65      0.62      1501
           3       0.68      0.61      0.65       644
           4       0.88      0.82      0.85       640
           5       0.76      0.71      0.73      1319
           6       0.69      0.50      0.58      1209
           7       0.82      0.66      0.73       995
           8       0.83      0.74      0.78      1314
           9       0.90      0.80      0.85       636

   micro avg       0.76      0.68      0.72      9952
   macro avg       0.78      0.68      0.72      9952
weighted avg       0.77      0.68      0.71      9952
 samples avg       0.76      0.71      0.73      9952



/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.10      0.06      0.07        17
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         9
           4       1.00      0.14      0.25         7
           5       0.82      0.79      0.80        42
           6       0.00      0.00      0.00         2
           7       1.00      0.20      0.33         5
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         7
          10       0.60      0.38      0.46        24
          11       0.67      0.18      0.29        11
          12       0.00      0.00      0.00         5
          13       1.00      0.25      0.40         4
          14       0.50      0.10      0.17        10
          15       0.34      0.29      0.31        66
          16       0.40      0.26      0.32        23
          17       0.00    

/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79        76
           1       0.76      0.46      0.57        35
           2       0.50      0.55      0.52       103
           3       0.55      0.41      0.47        39
           4       0.73      0.59      0.66        37
           5       0.65      0.63      0.64        86
           6       0.71      0.43      0.53        91
           7       0.78      0.58      0.67        72
           8       0.76      0.72      0.74        90
           9       0.81      0.68      0.74        50

   micro avg       0.69      0.59      0.64       679
   macro avg       0.71      0.58      0.63       679
weighted avg       0.70      0.59      0.64       679
 samples avg       0.67      0.62      0.64       679



/home/venkat/BADRI/badrienv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
